In [ ]:
# import the needed packages
from result_fusion import Fusion
from result_fusion import Fusion2

In [ ]:
# define the folder path and parameters
# train2 is the folder of the generated mask, preciser
# train is the folder of the generated mask, default sam params
mask_folder = '/media/ywh/1/yanweihao/projects/segmentation/segment-anything/outputs/cityscapes/train2'
# the path to the model prediction
segmentation_folder = '/media/ywh/1/yanweihao/projects/uda/DAFormer/work_dirs/local-exp7/230522_2312_gta2cs_dacs_a999_fdthings_rcs001_cpl_daformer_sepaspp_mitb5_poly10warm_s0_ea659/trainid'
# the path to the original image
image_folder = '/media/ywh/1/yanweihao/dataset/cityscapes_original/gtFine_trainvaltest/leftImg8bit/train_all'
# 
mix_ratio = 0.5
# 
resize_ratio = 0.5
# 
output_folder = 'outputs/cityscapes/debug'
# 
mask_suffix = '.png'
# 
segmentation_suffix ='trainID.png'
# 
segmentation_suffix_noimg=False
# 
fusion_mode = 1
# 
sam_classes = [5, 6, 7, 11, 12, 13, 14, 15, 16, 17, 18]
# 
shrink_num = 2

In [ ]:
fusion = Fusion2(mask_folder, segmentation_folder, image_folder, mix_ratio, resize_ratio, output_folder, mask_suffix, segmentation_suffix, segmentation_suffix_noimg, fusion_mode, sam_classes, shrink_num)